# Data extraction via Spotify API

In this notebook lies the code used to extract metadata from the songs that are going to be used to train the cluster algorithm and where song recommendation are going to come from.

To achieve gathering a very genre rich dataset track features have been manually extracted by calling different playlist in each of the following 13 genres:
- Country
- Pop
- Hip Hop
- R&B
- Jazz
- Blues
- Classical
- Latin
- Chill
- Workout
- Party
- Dance
- Disco

In [2]:
#These are the playlist uri's used to build the build the data set.

#Country playlists
['37i9dQZF1DWZBCPUIUs2iR', '1mJhSx6aYQmINsZ8dG4gzU', '4EKxAJeKAwzd0ifNsFOUYo', '4TgQnrfeA84A2uSBBnXPiw']

#Pop playlists
['3iWdDGK2cue86Tb7wufAFU', '6ihgnlqxvYZfc5LbBKS4Bt', '3k42316EoqW5fE8YjVpltu', '37i9dQZF1DWXJyjYpHunCf']

#HipHop playlists
['0weizyV5WNZP3tvfXWVfmg', '3RcRK9HGTAm9eLW1LepWKZ', '7oe9PHYHNshF3oc3X0IIdy', '6RIngYgeaogaJlbojgSFSX', '5DZ93TH6ABgWYRgdBNJs8O', '1L1VsYZJ55P731V7b8gFOk']

#R&B playlists
['37i9dQZF1DX04mASjTsvf0', '0bi6JVXt2xxjfYIr728Jsz', '37i9dQZF1DX4SBhb3fqCJd', '2dyKyLOiDnMJFm73ycNV4k']

#Jazz playlists
['7ABMovBcmt0PaZ9uRQ10Vv', '37i9dQZF1DXbITWG1ZJKYt', '37i9dQZF1DWV7EzJMK2FUI', '5EUhooJ7cFHScZcmfz66Qp', '37i9dQZF1DXdziGPHNE40t']

#Blues playlists
['37i9dQZF1DXd9rSDyQguIk', '5uBRWDOfL0Q0eyJ4IxdpxW' , '5TkTomPbQuSNDxdlWg2fCx', '1i9NKaGOEQsRmR392BqMxE', '1m6SO0I2Y82NCsAfmG7bVP']

#Classical playlists
['1h0CEZCm6IbFTbxThn6Xcs', '1CYNLCLR5ZH9tllxGPA4GS', '6wObnEPQ63a4kei1sEcMdH', '3BAtH3eZHdi4vfN6ZkpGiX']

#Latin playlists
['5JMihbVmqBAsdfvkVYX9Fv', '2fdCOYizvfG6JoH0zGQWB8','37i9dQZF1DX10zKzsJ2jva', '29Ah5RnWJ0YKxSzUpjk1kB', '00CtpUus1xPIBlHUTYo3yn', '1vIZxBcvgGHE9HXbQnzXYI']

#Chill playlists
['37i9dQZF1DWV1PBrIG2weG', '7ozIozDp260fjNOZy1yzRG', '37i9dQZF1DX4WYpdgoIcn6', '23tyPRHQ1kwG25OIlWBCbz', '6H2OZzeoaIdgL17NoF6jD5']

#Workout playlists
['2237sMNMlXS4wWLgdQ1UuV', '2PAGgoyhVYdC0q6vnb8qAL', '37i9dQZF1DWYp5sAHdz27Y', '37i9dQZF1DX32NsLKyzScr', '7mZZkjpyoY83wHbssEtzNF']

#Party playlists
['4tuReEKVsupVoQOldfIpiu', '37i9dQZF1DX7e8TjkFNKWH', '37i9dQZF1DWWylYLMvjuRG', '37i9dQZF1DX7F6T2n2fegs', '47ipPvW8GYxDzzS7vGemJb']

#Dance playlists
['04EKmJhBK8dLL6SONP5dyq', '37i9dQZF1DX0BcQWzuB7ZO', '37i9dQZF1DX8a1tdzq5tbM', '37i9dQZF1DX35X4JNyBWtb', '37i9dQZF1DXccH49bh52dB']

#Disco playlists
['5B3gb8HbvFm02ccQVH33IO', '3owJApDCnuxVg9nw6fqlSC', '2PzF3urLEgnQs4cmokAVHX', '52G9ij1Yrh1dVmXGXqVemZ', '370aiUDCl2Dm6BWgyE46YE']


['5B3gb8HbvFm02ccQVH33IO',
 '3owJApDCnuxVg9nw6fqlSC',
 '2PzF3urLEgnQs4cmokAVHX',
 '52G9ij1Yrh1dVmXGXqVemZ',
 '370aiUDCl2Dm6BWgyE46YE']

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
client_id = os.getenv("client_id")
client_secret = os.getenv("client_secret")

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [6]:
def get_playlist_tracks(username, playlist_id_list):
    df_big = []
    for playlist_id in playlist_id_list:
        results = sp.user_playlist_tracks(username,playlist_id)
        tracks = results['items']
        while results['next']:
            results = sp.next(results)
            tracks.extend(results['items'])
        results = tracks    

        playlist_tracks_id = []
        playlist_tracks_titles = []
        playlist_tracks_artists = []
        playlist_tracks_first_artists = []
        playlist_tracks_first_release_date = []
        playlist_tracks_popularity = []

        for i in range(len(results)):
            
            if i == 0:
                playlist_tracks_id = results[i]['track']['id']
                playlist_tracks_titles = results[i]['track']['name']
                playlist_tracks_first_release_date = results[i]['track']['album']['release_date']
                playlist_tracks_popularity = results[i]['track']['popularity']

                artist_list = []
                for artist in results[i]['track']['artists']:
                    artist_list= artist['name']
                playlist_tracks_artists = artist_list

                features = sp.audio_features(playlist_tracks_id)
                features_df = pd.DataFrame(data=features, columns=features[0].keys())
                features_df['title'] = playlist_tracks_titles
                features_df['all_artists'] = playlist_tracks_artists
                features_df['popularity'] = playlist_tracks_popularity
                features_df['release_date'] = playlist_tracks_first_release_date
                features_df = features_df[['id', 'title', 'all_artists', 'popularity', 'release_date',
                                           'danceability', 'energy', 'key', 'loudness',
                                           'mode', 'acousticness', 'instrumentalness',
                                           'liveness', 'valence', 'tempo',
                                           'duration_ms', 'time_signature']]
                continue
            else:
                try:
                    playlist_tracks_id = results[i]['track']['id']
                    playlist_tracks_titles = results[i]['track']['name']
                    playlist_tracks_first_release_date = results[i]['track']['album']['release_date']
                    playlist_tracks_popularity = results[i]['track']['popularity']
                    artist_list = []
                    for artist in results[i]['track']['artists']:
                        artist_list= artist['name']
                    playlist_tracks_artists = artist_list
                    features = sp.audio_features(playlist_tracks_id)
                    new_row = {'id':[playlist_tracks_id],
                   'title':[playlist_tracks_titles],
                   'all_artists':[playlist_tracks_artists],
                   'popularity':[playlist_tracks_popularity],
                   'release_date':[playlist_tracks_first_release_date],
                   'danceability':[features[0]['danceability']],
                   'energy':[features[0]['energy']],
                   'key':[features[0]['key']],
                   'loudness':[features[0]['loudness']],
                   'mode':[features[0]['mode']],
                   'acousticness':[features[0]['acousticness']],
                   'instrumentalness':[features[0]['instrumentalness']],
                   'liveness':[features[0]['liveness']],
                   'valence':[features[0]['valence']],
                   'tempo':[features[0]['tempo']],
                   'duration_ms':[features[0]['duration_ms']],
                   'time_signature':[features[0]['time_signature']]
                   }

                    dfs = [features_df, pd.DataFrame(new_row)]
                    
                    
                    df_big = df_big + [pd.DataFrame(new_row)]
                except:
                    continue
    features_df_big = pd.concat(df_big, ignore_index = True)       
    return features_df_big

In [13]:
df = get_playlist_tracks('spotify', ['5B3gb8HbvFm02ccQVH33IO', '3owJApDCnuxVg9nw6fqlSC', '37i9dQZF1DWZBCPUIUs2iR', '52G9ij1Yrh1dVmXGXqVemZ', '370aiUDCl2Dm6BWgyE46YE'])
df

,id,title,all_artists,popularity,release_date,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,5ubvP9oKmxLUVq506fgLhk,Stayin Alive,Bee Gees,77,2020-12-11,0.702,0.773,10,-9.256,0,0.0288,0.008710,0.1470,0.953,103.564,285373,4
1,2LScqpywMqGcnum6nNaxXX,"Yes Sir, I Can Boogie",Baccara,59,1998,0.454,0.769,2,-7.653,1,0.1270,0.000003,0.1750,0.491,122.531,273867,4
2,3K7Q9PHUWPTaknlbFPThn2,Celebration,Kool & The Gang,69,1980-09-29,0.885,0.664,1,-11.795,1,0.1450,0.048300,0.0767,0.906,121.632,298867,4
3,2Foc5Q5nqNiosCNqttzHof,Get Lucky (feat. Pharrell Williams & Nile Rodg...,Nile Rodgers,79,2013-04-19,0.794,0.811,6,-8.966,0,0.0426,0.000001,0.1010,0.862,116.047,248413,4
4,2xiOdusRnZezQok1RgLNeS,You Should Be Dancing,Bee Gees,66,1976-01-01,0.681,0.716,0,-10.911,1,0.0182,0.170000,0.3370,0.963,123.009,256493,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802,4qM461TqtpnP4GLRIXwEnW,Sussudio - 2016 Remaster,Phil Collins,64,1985-01-25,0.714,0.935,5,-5.195,1,0.0529,0.093400,0.0515,0.725,121.232,263107,4
803,1ot6jEe4w4hYnsOPjd3xKQ,I'm So Excited,The Pointer Sisters,70,1982-02-01,0.590,0.868,5,-5.832,0,0.0962,0.000322,0.4420,0.798,184.348,230640,4
804,0Dm43YLUlGdePpSbI1ct8h,Part-Time Lover,Stevie Wonder,67,1985-09-13,0.662,0.624,10,-13.606,0,0.1650,0.000000,0.0674,0.961,174.513,252560,4
805,2fjJt3OQ8MPaZ3LMzAWiTA,Typical Male,Tina Turner,49,1986-01-10,0.762,0.745,1,-7.772,0,0.0172,0.013700,0.0612,0.794,106.013,256827,4


In [31]:
df.shape

(812, 17)

In [33]:
df.to_csv('./data/mix_5.csv', index = False)

In [15]:
df1 = get_playlist_tracks('spotify', ['37i9dQZF1DWZBCPUIUs2iR', '1mJhSx6aYQmINsZ8dG4gzU', '4EKxAJeKAwzd0ifNsFOUYo', '4TgQnrfeA84A2uSBBnXPiw'])
df2 = get_playlist_tracks('spotify', ['3iWdDGK2cue86Tb7wufAFU', '6ihgnlqxvYZfc5LbBKS4Bt', '3k42316EoqW5fE8YjVpltu', '37i9dQZF1DWXJyjYpHunCf'])
df3 = get_playlist_tracks('spotify', ['0weizyV5WNZP3tvfXWVfmg', '3RcRK9HGTAm9eLW1LepWKZ', '7oe9PHYHNshF3oc3X0IIdy', '6RIngYgeaogaJlbojgSFSX', '5DZ93TH6ABgWYRgdBNJs8O', '1L1VsYZJ55P731V7b8gFOk'])
df4 = get_playlist_tracks('spotify', ['37i9dQZF1DX04mASjTsvf0', '0bi6JVXt2xxjfYIr728Jsz', '37i9dQZF1DX4SBhb3fqCJd', '2dyKyLOiDnMJFm73ycNV4k'])
df5 = get_playlist_tracks('spotify', ['7ABMovBcmt0PaZ9uRQ10Vv', '37i9dQZF1DXbITWG1ZJKYt', '37i9dQZF1DWV7EzJMK2FUI', '5EUhooJ7cFHScZcmfz66Qp', '37i9dQZF1DXdziGPHNE40t'])
df6 = get_playlist_tracks('spotify', ['37i9dQZF1DXd9rSDyQguIk', '5uBRWDOfL0Q0eyJ4IxdpxW' , '5TkTomPbQuSNDxdlWg2fCx', '1i9NKaGOEQsRmR392BqMxE', '1m6SO0I2Y82NCsAfmG7bVP'])
df7 = get_playlist_tracks('spotify', ['1h0CEZCm6IbFTbxThn6Xcs', '1CYNLCLR5ZH9tllxGPA4GS', '6wObnEPQ63a4kei1sEcMdH', '3BAtH3eZHdi4vfN6ZkpGiX'])
df8 = get_playlist_tracks('spotify', ['5JMihbVmqBAsdfvkVYX9Fv', '2fdCOYizvfG6JoH0zGQWB8','37i9dQZF1DX10zKzsJ2jva', '29Ah5RnWJ0YKxSzUpjk1kB', '00CtpUus1xPIBlHUTYo3yn', '1vIZxBcvgGHE9HXbQnzXYI'])
df9 = get_playlist_tracks('spotify', ['37i9dQZF1DWV1PBrIG2weG', '7ozIozDp260fjNOZy1yzRG', '37i9dQZF1DX4WYpdgoIcn6', '23tyPRHQ1kwG25OIlWBCbz', '6H2OZzeoaIdgL17NoF6jD5'])
df10 = get_playlist_tracks('spotify', ['2237sMNMlXS4wWLgdQ1UuV', '2PAGgoyhVYdC0q6vnb8qAL', '37i9dQZF1DWYp5sAHdz27Y', '37i9dQZF1DX32NsLKyzScr', '7mZZkjpyoY83wHbssEtzNF'])
df11 = get_playlist_tracks('spotify', ['4tuReEKVsupVoQOldfIpiu', '37i9dQZF1DX7e8TjkFNKWH', '37i9dQZF1DWWylYLMvjuRG', '37i9dQZF1DX7F6T2n2fegs', '47ipPvW8GYxDzzS7vGemJb'])
df12 = get_playlist_tracks('spotify', ['04EKmJhBK8dLL6SONP5dyq', '37i9dQZF1DX0BcQWzuB7ZO', '37i9dQZF1DX8a1tdzq5tbM', '37i9dQZF1DX35X4JNyBWtb', '37i9dQZF1DXccH49bh52dB'])
df13 = get_playlist_tracks('spotify', ['5B3gb8HbvFm02ccQVH33IO', '3owJApDCnuxVg9nw6fqlSC', '37i9dQZF1DWZBCPUIUs2iR', '52G9ij1Yrh1dVmXGXqVemZ', '370aiUDCl2Dm6BWgyE46YE'])



In [9]:
df1.to_csv('./data/mix_1.csv')
df2.to_csv('./data/mix_2.csv')
df3.to_csv('./data/mix_3.csv')
df4.to_csv('./data/mix_4.csv')
df5.to_csv('./data/mix_5.csv')
df6.to_csv('./data/mix_6.csv')
df7.to_csv('./data/mix_7.csv')
df8.to_csv('./data/mix_8.csv')
df9.to_csv('./data/mix_9.csv')
df10.to_csv('./data/mix_10.csv')
df11.to_csv('./data/mix_11.csv')
df12.to_csv('./data/mix_12.csv')
df13.to_csv('./data/mix_13.csv')


In [17]:
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13])

df.shape

(12202, 17)

In [18]:
df.head()

,id,title,all_artists,popularity,release_date,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,6sxptembJVty4sNtcPMAVz,Man! I Feel Like A Woman!,Shania Twain,75,1997-11-04,0.657,0.639,10,-7.527,1,0.5810,0.000000,0.2780,0.674,124.882,234493,4
1,6iy4PoAuZBMvtrlDX4VxC7,Check Yes Or No,George Strait,69,1995-09-12,0.708,0.524,5,-12.644,1,0.0551,0.000003,0.0371,0.690,111.482,200667,4
2,4w3tQBXhn5345eUXDGBWZG,9 to 5,Dolly Parton,75,1980,0.554,0.783,6,-5.852,1,0.4160,0.000002,0.6310,0.813,105.390,162467,4
3,5KqldkCunQ2rWxruMEtGh0,The Gambler,Kenny Rogers,73,1978-11-20,0.671,0.501,4,-13.119,1,0.3420,0.000000,0.1940,0.860,87.040,214880,4
4,6xHI9KjUjYT0FPtGO8Mxa1,Hurricane,Luke Combs,73,2017-06-02,0.464,0.813,10,-6.185,1,0.0153,0.000000,0.2540,0.515,75.977,222533,4


In [19]:
#Saving the dataset
df.to_csv('./data/DATA.csv', index = False)